In [82]:
import sympy as sp

h = sp.symbols("h", real=True)

In [83]:
# define desired polynomial degree and interpolation point on [-h / 2, h / 2]
p = 7
x = h / 2 * sp.sqrt(525 + 70 * sp.sqrt(30)) / 35

In [84]:
def derive_stencil(x_interp, l, r):
    """
    Derive the conservative interpolation stencil for a given interpolation
    point `x_interp` and left/right stencil widths `l` and `r`.

    {U_{-l}, U_{-l+1}, ..., U_{r}} -> u(x_interp)
    """
    x = sp.symbols("x", real=True)

    interfaces = [-l * h - h / 2] + [i * h + h / 2 for i in range(-l, r + 1)]
    ninterfaces = len(interfaces)

    averages = sp.symbols(" ".join(f"U_{i}" for i in range(-l, r + 1)), real=True)
    if not isinstance(averages, tuple):
        averages = (averages,)

    lagrange_polys, cum_masses = [], []
    for j in range(ninterfaces):
        lagrange_poly_j = 1
        for k in range(ninterfaces):
            if j == k:
                continue
            lagrange_poly_j *= (x - interfaces[k]) / (interfaces[j] - interfaces[k])
        lagrange_polys.append(lagrange_poly_j)

        cum_masses.append(sum(averages[i] for i in range(j)))

    P = sum(lagrange_polys[j] * cum_masses[j] for j in range(ninterfaces))
    u = h * sp.diff(P, x)

    stencil = sp.expand(u.subs(x, x_interp))
    return stencil

In [85]:
# derive the symmetric stencil
if (p + 1) % 2 == 0:
    stencil1 = derive_stencil(x, p // 2, (p + 1) // 2)
    stencil2 = derive_stencil(x, (p + 1) // 2, p // 2)
    stencil = (stencil1 + stencil2) / 2
else:
    stencil = derive_stencil(x, (p + 1) // 2, (p + 1) // 2)

In [86]:
# sort the coefficients by index and print
index_term_map = {int(str(s).strip("U_")): s for s in stencil.free_symbols}
sorted_indices = sorted(index_term_map)

print(
    f"u(x={x}) = "
    + " + ".join(f"(w_{i} * {index_term_map[i]})" for i in sorted_indices),
    end="\n\n",
)
print("\n".join(f"w_{i}: {stencil.coeff(index_term_map[i])}" for i in sorted_indices))

u(x=h*sqrt(70*sqrt(30) + 525)/70) = (w_-4 * U_-4) + (w_-3 * U_-3) + (w_-2 * U_-2) + (w_-1 * U_-1) + (w_0 * U_0) + (w_1 * U_1) + (w_2 * U_2) + (w_3 * U_3) + (w_4 * U_4)

w_-4: -3177*sqrt(30)*sqrt(70*sqrt(30) + 525)/2689120000 + 37831*sqrt(70*sqrt(30) + 525)/605052000
w_-3: -798883*sqrt(70*sqrt(30) + 525)/1210104000 + 307/1646400 + 307*sqrt(30)/2744000 + 143599*sqrt(30)*sqrt(70*sqrt(30) + 525)/12101040000
w_-2: -91033*sqrt(30)*sqrt(70*sqrt(30) + 525)/1728720000 - 1971*sqrt(30)/1372000 - 657/274400 + 9119*sqrt(70*sqrt(30) + 525)/2701125
w_-1: -700963*sqrt(70*sqrt(30) + 525)/57624000 + 128693*sqrt(30)*sqrt(70*sqrt(30) + 525)/1728720000 + 6521/329280 + 6521*sqrt(30)/548800
w_0: 79423/82320 - 2897*sqrt(30)/137200
w_1: -128693*sqrt(30)*sqrt(70*sqrt(30) + 525)/1728720000 + 6521/329280 + 6521*sqrt(30)/548800 + 700963*sqrt(70*sqrt(30) + 525)/57624000
w_2: -9119*sqrt(70*sqrt(30) + 525)/2701125 - 1971*sqrt(30)/1372000 - 657/274400 + 91033*sqrt(30)*sqrt(70*sqrt(30) + 525)/1728720000
w_3: -143599*sq